In [72]:
from selenium import webdriver
url = 'https://www.latam.com/es_pe/apps/personas/booking?fecha1_dia=12&fecha1_anomes=2020-10&from_city1=LIM&to_city1=SCL&ida_vuelta=ida_vuelta&fecha2_dia=19&fecha2_anomes=2020-10&from_city2=SCL&to_city2=LIM&cabina=Y&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [85]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import time

In [77]:
def getBasicTravelInfo(travel):
    departure = travel.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    arrival = travel.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    duration = travel.find_element_by_xpath('.//span[@class="duration"]/time').text
    return {'departure': departure,
            'arrival': arrival,
            'duration': duration
            }


In [78]:
def clickButtonEscalas(travel):
    button=travel.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
    button.click()


In [79]:
def getItinerary(driver):
    itinerary = {}
    itinerario = driver.find_element_by_xpath('//div[@class="sc-hZSUBg gfeULV"]')
    horarios = itinerario.find_elements_by_xpath('//div[@class="sc-cLQEGU hyoued"]')
    for index,horario in enumerate(horarios):
        departure_airport = horario.find_elements_by_xpath('.//abbr[@class="sc-hrWEMg hlCkST"]')[0].text
        departure_time = horario.find_elements_by_xpath('.//time[@class="sc-RefOD libzvk"]')[0].get_attribute('datetime')
        departure_airport_full_name = horario.find_elements_by_xpath('.//span[@class="sc-hXRMBi gVvErD"]')[0].text
        arrival_airport = horario.find_elements_by_xpath('.//abbr[@class="sc-hrWEMg hlCkST"]')[1].text
        arrival_time = horario.find_elements_by_xpath('.//time[@class="sc-RefOD libzvk"]')[1].get_attribute('datetime')
        arrival_airport_full_name = horario.find_elements_by_xpath('.//span[@class="sc-hXRMBi gVvErD"]')[1].text
        flight_time = horario.find_element_by_xpath('.//div[@class="sc-eXEjpC fqUnRK"]/span/time').get_attribute('datetime')
        ariplane_model = horario.find_element_by_xpath('.//span[@class="sc-gzOgki uTyOl"]').text
        itinerary[index + 1] = {
            'departure_airport' : departure_airport,
            'departure_time' : departure_time,
            'departure_airport_full_name' : departure_airport_full_name,
            'arrival_airport': arrival_airport,
            'arrival_time': arrival_time,
            'arrival_airport_full_name': arrival_airport_full_name,
            'flight_time': flight_time,
            'airplane_model': ariplane_model,
            }

    return itinerary

In [80]:
def closeModal(driver):
    close_button = driver.find_element_by_xpath('//div[@class="modal-header sc-dnqmqq cGfTsx"]/button')
    close_button.click()

In [81]:
def clickOnFlight(flight):
    flight.click()

In [83]:
def getPrices(flight):
    rates_elements = flight.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    rates = []
    for rate_element in rates_elements:
        rate_name = rate_element.find_element_by_xpath('.//input[@name="selected-fare"]').get_attribute('id')
        money_symbol = rate_element.find_element_by_xpath('.//span[@class="currency-symbol"]').text
        value = rate_element.find_element_by_xpath('.//span[@class="value"]/span').text
        rates.append({rate_name: 
            {
                'money_symbol': money_symbol,
                'value': value
            }
        })
    return rates

In [88]:
def get_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    info = []
    #removeremos la notificacion
    try:
        notification = WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@id="onesignal-slidedown-container"]')))
        if(notification):
            no_notification_button = driver.find_element_by_xpath('//button[@id="onesignal-slidedown-cancel-button"]')
            no_notification_button.click()
        print('Cerramos la notificacion')
    except TimeoutException:
        print('La notificacion demoro en salir, procederemos')

    #removeremos el modal
    try:
        modal = WebDriverWait(driver, 10).until(expected_conditions.presence_of_element_located((By.XPATH, '//div[@class="lightbox-overlay is-active"]')))
        if(modal):
            close_modal_button = driver.find_element_by_xpath('//div[@class="lightbox-top"]/span') #usan un span para el boton
            close_modal_button.click()
        print('Cerramos la notificacion')
    except TimeoutException:
        print('El modal demoro o no salio, procedemos')
    for vuelo in vuelos:
        tiempos = getBasicTravelInfo(vuelo)
        clickButtonEscalas(vuelo)
        escalas = getItinerary(driver)
        closeModal(driver)
        clickOnFlight(vuelo) #Opens to get escalas
        precios = getPrices(vuelo)
        clickOnFlight(vuelo) #Closes to proceed
        info.append({
            'precios': precios,
            'tiempos': tiempos,
            'escalas': escalas
        })

    return info


In [91]:
options = webdriver.ChromeOptions()
options.add_argument('--incognito')
driver = webdriver.Chrome(executable_path='./chromedriver', options=options)
driver.get(url)
# Introducir una demora
delay = 10
try:
    vuelo = WebDriverWait(driver, delay).until(expected_conditions.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La pagina termino de cargar')
    info_vuelos = get_info(driver)
except TimeoutException:
    print('La pagina tardo demasiado en cargar')

driver.close()

La pagina termino de cargar
Se encontraron 2 vuelos.
Cerramos la notificacion
El modal demoro o no salio, procedemos


In [90]:
info_vuelos


[{&#39;precios&#39;: [{&#39;BASIC&#39;: {&#39;money_symbol&#39;: &#39;&#39;, &#39;value&#39;: &#39;&#39;}},
   {&#39;LIGHT&#39;: {&#39;money_symbol&#39;: &#39;&#39;, &#39;value&#39;: &#39;&#39;}},
   {&#39;PLUS&#39;: {&#39;money_symbol&#39;: &#39;&#39;, &#39;value&#39;: &#39;127&#39;}},
   {&#39;TOP&#39;: {&#39;money_symbol&#39;: &#39;US$&#39;, &#39;value&#39;: &#39;165&#39;}}],
  &#39;tiempos&#39;: {&#39;departure&#39;: &#39;18:00&#39;,
   &#39;arrival&#39;: &#39;23:34&#39;,
   &#39;duration&#39;: &#39;3h 34min&#39;},
  &#39;escalas&#39;: {1: {&#39;departure_airport&#39;: &#39;LIM&#39;,
    &#39;departure_time&#39;: &#39;18:00&#39;,
    &#39;departure_airport_full_name&#39;: &#39;J Chávez Intl.&#39;,
    &#39;arrival_airport&#39;: &#39;SCL&#39;,
    &#39;arrival_time&#39;: &#39;23:34&#39;,
    &#39;arrival_airport_full_name&#39;: &#39;A. Merino Benítez Intl.&#39;,
    &#39;flight_time&#39;: &#39;3:34&#39;,
    &#39;airplane_model&#39;: &#39;Airbus 320-200&#39;}}},
 {&#39;precios&#39;: